In [111]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [126]:
url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
pages = 20
data = []

In [127]:
for page in range(1,pages+1):
    url = url + str(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    
    products = soup.find_all('div',{'data-component-type':'s-search-result'})
    
    for product in products:
        product_data = {}
        
        # Extract product URL
        product_link = product.find('a', {'class': 'a-link-normal s-no-outline'})['href']
        product_url = "https://www.amazon.in" + product_link
        product_data['Product URL'] = product_url
        
        # Extract product name
        product_name = product.find('span', {'class': 'a-size-medium a-color-base a-text-normal'}).text.strip()
        product_data['Product Name'] = product_name
        
        # Extract product price
        product_price = product.find('span',{'class':'a-price-whole'}).text.strip()
        product_data['Product Price'] = product_price
        
        # Extract product rating
        product_rating = product.find('span',{'class': 'a-icon-alt'})
        product_rating =  product_rating.text.strip() if product_rating else "N/A"
        product_data['Rating'] = product_rating
        
        # Extract number of product reviews
        product_reviews = product.find('span', {'class': 'a-size-base'})
        product_reviews = product_reviews.text.strip() if product_reviews else "0"
        product_data['Number of Reviews'] = product_reviews
        
        data.append(product_data)

In [128]:
# to scrap additional details from product URLS
for i, product in enumerate(data):
    url = url+str(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    
    # Extract  description
    description = soup.find('div', {'id': 'feature-bullets'}).text.strip() if soup.find('div', {'id': 'feature-bullets'}) else "N/A"
    data[i]['Description'] = description
    
#     # Extract product ASIN
#     product_asin = soup.find('li', text='ASIN').find('span', {'class': 'a-text-bold'}).find_next('span').text.strip() if soup.find('li', text='ASIN') else "N/A"
#     data[i]['ASIN'] = product_asin
    asin_element = soup.find('li', string=lambda text: text and 'ASIN' in text)
    asin = asin_element.find('span', class_='a-list-item').next_sibling.strip() if asin_element else "N/A"
    data[i]['ASIN'] = asin
    
    # Extract product description
    product_description = soup.find('div', {'id': 'feature-bullets'}).text.strip() if soup.find('div', {'id': 'feature-bullets'}) else "N/A"
    data[i]['Product Description'] = product_description

#     # Extract manufacturer
#     product_manufacturer = soup.find('span', {'class': 'a-list-item'})
#     product_manufacturer = product_manufacturer.text.strip() if product_manufacturer else "N/A"
#     data[i]['Manufacturer'] = product_manufacturer

    # Extract manufacturer
    manufacturer = soup.find('span', text='Manufacturer').find_next('span').text.strip() if soup.find('span', text='Manufacturer') else "N/A"
    data[i]['Manufacturer'] = manufacturer

In [130]:
# for i, product in enumerate(data):
url = url+str(page)
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')
    
    
#     asin = soup.find('div', {'id': 'detailBullets_feature_div'}).find('ul',{'class':'a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list'}).find('span',{'class':'a-list-item'}).find('span',{'class':'a-text-bold'}).text if soup.find('div', {'id': 'detailBullets_feature_div'}) else "N/A" 
asin = soup.find('div',attrs={'id':'detailBulletsReverseInterleaveContainer_feature_v2'})
        
#     # Extract product ASIN
#     # Find the <div> with id 'detailBullets_feature_div'
#     detail_bullets_div = soup.find('div', {'id': 'detailBullets_feature_div'})

# # Find the <ul> within the 'detailBullets_feature_div'
#     ul_element = detail_bullets_div.find('ul', class_='a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list')

# # Find the <li> containing ASIN
#     asin_li = ul_element.find('li', string=lambda text: text and 'ASIN' in text)

# # Extract the ASIN from the <span> tag within the <li>
#     asin = asin_li.find('span', class_='a-list-item').find_next('span').text.strip() if asin_li else "N/A"

# # Assign ASIN to the dictionary
#     data[i]['ASIN'] = asin   


In [133]:
# Export data to CSV
df = pd.DataFrame(data)
df.to_csv('amazon_products.csv', index=False)